In [ ]:
import dash
from dash import dcc
from dash import html
import plotly.graph_objects as go
import plotly.express as px
from dash.dependencies import Input, Output



In [2]:
app=dash.Dash()
df=px.data.stocks()

In [3]:
def stock_prices():
    fig=go.Figure([go.Scatter(x=df['date'], y=df['GOOG'],
                              line=dict(color='firebrick', width=4),name='Google')])
    fig.update_layout(title='Prices over time',
                     xaxis_title='Dates', yaxis_title='Prices')
    return fig

In [4]:
app.layout=html.Div(id='parent',
                    children=[html.H1(id='H1', 
                                      children='Styling using html components',
                                      style={'testAlign':'center', 'marginTop':40,'marginBottom':40}),
                              dcc.Dropdown(id='dropdown',options=[{'label':'GOOGLE', 'value':'GOOG'},{'label':'Apple', 'value':'APPL'},{'label':'Amazon', 'value':'AMZN'}],value='GOOG'),
                             dcc.Graph(id='line_plot', figure=stock_prices())])


In [5]:
@app.callback(Output(component_id='line_plot', component_property='figure'),
             [Input(component_id='dropdown', component_property='value')])
def graph_update(dropdown_value):
    print(dropdown_value)
    fig=go.Figure([go.Scatter(x=df['date'],y=df['{}'.format(dropdown_value)],
                             line=dict(color='firebrick', width=4))])
    fig.update_layout(title='Stock prices over time',
                     xaxis_title='Date',
                     yaxis_title='Prices')
    return fig


In [6]:
if __name__ == '__main__': 
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [10/Nov/2022 22:20:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2022 22:20:30] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2022 22:20:30] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2022 22:20:30] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [10/Nov/2022 22:20:30] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [10/Nov/2022 22:20:30] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [10/Nov/2022 22:20:30] "GET /_favicon.ico?v=2.7.0 HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2022 22:20:30] "POST /_dash-update-component HTTP/1.1" 200 -


GOOG


In [1]:
exit

# Stand up heat map

In [1]:
import dash
from dash import dcc
from dash import html
import plotly.graph_objects as go
import plotly.express as px
from dash.dependencies import Input, Output

import mysql.connector as mysql
import pandas as pd
from mysql.connector import Error
import datetime as dt


### get data

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
exec(open('/Users/franciscoavalosjr/Desktop/basketball-creds.py').read())

connection=mysql.connect(host=sports_db_admin_host,
                        database=sports_db_admin_db,
                        user=sports_db_admin_user,
                        password=sports_db_admin_pw,
                        port=3306)

In [5]:
if connection.is_connected():
    cursor=connection.cursor()
    cursor.execute('SELECT MTS.*, C.week_starting_monday, C.week_ending_sunday FROM basketball.my_team_stats MTS JOIN basketball.calendar C ON MTS.date=C.day;')
    myteam_df=cursor.fetchall()
    myteam_df=pd.DataFrame(myteam_df, columns=cursor.column_names)

In [6]:
if connection.is_connected():
    cursor=connection.cursor()
    cursor.execute('SELECT * FROM basketball.live_free_agents;')
    fa_df=cursor.fetchall()
    fa_df=pd.DataFrame(fa_df, columns=cursor.column_names)

In [7]:
if(connection.is_connected()):
    cursor.close()
    connection.close()
    print('Script finished - MySQL connection is closed')
else:
    print('MySQL already closed')

Script finished - MySQL connection is closed


In [8]:
myteam_df['total_rebounds']=myteam_df['offensive_rebounds']+myteam_df['defensive_rebounds']
myteam_df['minutes_played']=myteam_df['seconds_played']/60

fa_df['total_rebounds']=fa_df['offensive_rebounds']+fa_df['defensive_rebounds']
fa_df['minutes_played']=fa_df['seconds_played']/60

In [9]:
# myteam_df.sort_values('date', ascending=False)
# myteam_df.shape
fa_df.head()

,name,date,team,location,opponent,outcome,active,seconds_played,made_field_goals,attempted_field_goals,made_three_point_field_goals,attempted_three_point_field_goals,made_free_throws,attempted_free_throws,offensive_rebounds,defensive_rebounds,assists,steals,blocks,turnovers,personal_fouls,points_scored,game_score,plus_minus,total_rebounds,minutes_played
0,Aaron Holiday,2022-10-19,Team.ATLANTA_HAWKS,Location.HOME,Team.HOUSTON_ROCKETS,Outcome.WIN,1,1082,2,5,1,2,0,0,0,0,1,3,0,2,1,5,3.60,-6,0,18.033333
1,Aaron Holiday,2022-10-21,Team.ATLANTA_HAWKS,Location.HOME,Team.ORLANDO_MAGIC,Outcome.WIN,1,862,2,3,0,0,0,0,0,2,1,0,0,0,2,4,3.20,0,2,14.366667
2,Aaron Holiday,2022-10-23,Team.ATLANTA_HAWKS,Location.HOME,Team.CHARLOTTE_HORNETS,Outcome.LOSS,1,822,0,4,0,2,2,2,0,0,0,1,0,1,1,2,-1.20,-22,0,13.700000
3,Aaron Holiday,2022-10-26,Team.ATLANTA_HAWKS,Location.AWAY,Team.DETROIT_PISTONS,Outcome.WIN,1,1067,4,8,2,2,0,0,2,0,2,0,1,1,1,10,8.10,3,2,17.783333
4,Aaron Holiday,2022-10-28,Team.ATLANTA_HAWKS,Location.AWAY,Team.DETROIT_PISTONS,Outcome.WIN,1,1672,5,6,1,1,3,3,1,1,3,2,1,1,4,14,15.00,17,2,27.866667


In [13]:
# myteam_df.head()
cols=['made_field_goals', 'made_three_point_field_goals','made_free_throws', 
      'total_rebounds', 'offensive_rebounds', 'defensive_rebounds', 'assists', 
      'steals', 'blocks', 'turnovers', 'personal_fouls', 'points_scored', 'minutes_played'
     ]

focus_value='made_field_goals'
calc='sum'
player_sample=20
days_ago=7

today=dt.date.today()
days_back=today-dt.timedelta(days=days_ago)
# calc_presentation='Standard Deviation' if calc=='std' else 'Average' if calc=='Mean' else 'Total'


# sn.set(rc={'figure.figsize':(15,8)})

df_query=fa_df.query("date >= @days_back")
output=df_query.groupby(['name'])[cols].agg(calc).reset_index().sort_values(by=[focus_value],ascending=False).head(player_sample)

output.set_index(['name'], inplace=True, drop=True, append=False)
output.reset_index(inplace=False)
cmap='Spectral_r'
# sn.heatmap(output.iloc[:,0:len(output)-1], annot=True, cmap=cmap)

# plt.suptitle(f'Past {days_ago} day(s) - {calc_presentation} Analysis')
# plt.title(f'{focus_value} leading')
# plt.show()

fig=px.imshow(output, text_auto=True)
fig.update_xaxes(side='top')
fig.layout.height=800
fig.layout.width=800
fig.show()


# output.shape





In [44]:
imps=[
 'made_field_goals',
 'made_three_point_field_goals',
 'made_free_throws',
 'offensive_rebounds',
 'defensive_rebounds',
 'assists',
 'steals',
 'blocks',
 'turnovers',
 'personal_fouls',
 'points_scored',
 'total_rebounds']
imps2=[
 'made_field_goals',
 'made_three_point_field_goals']

lst=[x for x in imps2 for a in imps if a in x]
lst

['made_field_goals', 'made_three_point_field_goals']

In [50]:
##### IM A TEST!!!!
# myteam_df.head()

cols=['made_field_goals', 'made_three_point_field_goals','made_free_throws', 
      'total_rebounds', 'offensive_rebounds', 'defensive_rebounds', 'assists', 
      'steals', 'blocks', 'turnovers', 'personal_fouls', 'points_scored', 'minutes_played'
     ]

imps=[
 'made_field_goals',
 'made_three_point_field_goals',
 'made_free_throws',
 'offensive_rebounds',
 'defensive_rebounds',
 'assists',
 'steals',
 'blocks',
 'turnovers',
 'personal_fouls',
 'points_scored',
 'total_rebounds']

focus_value='made_field_goals'
calc='sum'
player_sample=20
days_ago=14

today=dt.date.today()
days_back=today-dt.timedelta(days=days_ago)
# calc_presentation='Standard Deviation' if calc=='std' else 'Average' if calc=='Mean' else 'Total'

# import numpy as np
# def w_avg(df, values, weights):
#     d=df[values]
#     w=df[weights]
#     return (d*w).sum()/w.sum()

# def weighted(df, cols, weights='minutes_played'):
#     return pd.Series(np.average(df[cols], df[weights], axis=0), cols)


# sn.set(rc={'figure.figsize':(15,8)})

df_query=fa_df.query("date >= @days_back")
# df_query.groupby(['name']).apply(w_avg, 'points_scored', 'minutes_played')
# df_query.groupby(['name']).apply(w_avg, ['made_field_goals',
#                                          'made_three_point_field_goals'], 'minutes_played')
output=df_query.groupby(['name']).apply(lambda x: pd.Series([sum(x[v]*x.minutes_played)/sum(x.minutes_played) for v in imps]))
# output=pd.DataFrame(output)
# output.columns=imps
# output.set_index(['name'], inplace=True, drop=True, append=False)
# output.reset_index(inplace=False)
output

# [col for col in df_query.columns[:]]
# output=df_query.groupby(['name'])[cols].agg(calc).reset_index().sort_values(by=[focus_value],ascending=False).head(player_sample)

# output.set_index(['name'], inplace=True, drop=True, append=False)
# output.reset_index(inplace=False)
# output
# cmap='Spectral_r'
# # sn.heatmap(output.iloc[:,0:len(output)-1], annot=True, cmap=cmap)

# # plt.suptitle(f'Past {days_ago} day(s) - {calc_presentation} Analysis')
# # plt.title(f'{focus_value} leading')
# # plt.show()

# fig=px.imshow(output, text_auto=True)
# fig.update_xaxes(side='top')
# fig.layout.height=800
# fig.layout.width=800
# fig.show()


# # output.shape





,0,1,2,3,4,5,6,7,8,9,10,11
name,,,,,,,,,,,,
AJ Griffin,4.602544,1.212312,0.704460,1.144943,1.628612,0.788160,1.218436,0.000000,1.258794,1.317525,11.121859,2.773555
Aaron Holiday,1.648963,0.312132,0.379831,0.246353,1.484771,1.934221,0.875352,0.000000,0.660993,2.060660,3.989890,1.731124
Aaron Nesmith,0.801208,0.534139,0.534139,0.000000,2.406344,1.753172,0.732931,0.673414,1.079758,1.534139,2.670695,2.406344
Aaron Wiggins,2.995328,0.737754,1.215976,0.797438,2.456820,2.749962,0.493595,0.363075,0.664205,2.887867,7.944386,3.254258
Admiral Schofield,1.772786,0.425130,0.000000,0.449219,0.871745,1.347656,0.449219,0.000000,0.449219,2.267578,3.970703,1.320964
Alec Burks,5.000000,1.400158,4.799526,0.000000,2.000000,1.400158,0.400158,0.000000,0.400158,3.000000,16.199684,2.000000
Alex Caruso,1.568295,0.858872,0.599633,0.626972,3.706397,4.424681,1.342338,1.065809,1.816512,2.123622,4.595094,4.333369
Alex Len,2.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,1.000000
Amir Coffey,0.310434,0.258076,1.235425,0.421463,0.000000,1.073524,0.516153,0.000000,0.652061,1.832529,2.114371,0.421463


In [10]:
# short_df = output.iloc[:,0:len(output)-1]
short_df = output.iloc[:,0:len(output)]

In [11]:
short_df.shape

(20, 13)

In [12]:
# fig=px.imshow(short_df, text_auto=True)
# fig.layout.height=750
# fig.layout.width=750
# fig.show()

In [13]:
def player_stats():
    fig=px.imshow(short_df, text_auto=True)
    fig.update_xaxes(side='top')
    fig.layout.height=800
    fig.layout.width=800
    return fig


In [14]:
# player_stats()

In [15]:
app=dash.Dash()

In [16]:
app.layout=html.Div(children=[dcc.Graph(figure=player_stats(), id='line_plot'),
                              'How many days back?',
                              dcc.Input(id='my_input', value=7, type='integer'),
                              'Focus Field\n',
                             dcc.Dropdown(id='dropdown',
                                         options=[{'label':'Made Field Goals','value':'made_field_goals'},
                                                  {'label':'Made 3p Field Goals','value':'made_three_point_field_goals'},
                                                  {'label':'Made Free Throws','value':'made_free_throws'},
                                                  {'label':'Total Rebounds','value':'total_rebounds'},
                                                  {'label':'Offensive Rebounds','value':'offensive_rebounds'},
                                                  {'label':'Defensive Rebounds','value':'defensive_rebounds'},
                                                  {'label':'Assists','value':'assists'},
                                                  {'label':'Steals','value':'steals'},
                                                  {'label':'Blocks','value':'blocks'},
                                                  {'label':'Turnovers','value':'turnovers'},
                                                  {'label':'Personal Fouls','value':'personal_fouls'},
                                                  {'label':'Points','value':'points_scored'},
                                                  {'label':'Minutes Played','value':'minutes_played'}],
                                         value='made_field_goals'),
                              'Calculation type',
                              dcc.Dropdown(id='calculation', 
                                          options=[{'label':'Total', 'value':'sum'},
                                                   {'label':'Average(non-weighted))', 'value':'mean'},
                                                   {'label':'Standard Deviation', 'value':'std'}],
                                          value='sum'),
                              'Fields to display',
                             dcc.Checklist(id='displayed_fields', 
                                           options=['made_field_goals', 'made_three_point_field_goals',
                                                    'made_free_throws','total_rebounds', 'offensive_rebounds', 
                                                    'defensive_rebounds', 'assists','steals', 'blocks', 
                                                    'turnovers', 'personal_fouls', 'points_scored', 'minutes_played'],
                                          value=['made_field_goals', 'made_three_point_field_goals',
                                                    'made_free_throws','total_rebounds', 'offensive_rebounds', 
                                                    'defensive_rebounds', 'assists','steals', 'blocks', 
                                                    'turnovers', 'personal_fouls']),
                              'Number of players',
                             dcc.Input(id='top_n', value=5, type='integer')
                             ])



In [17]:
@app.callback(Output(component_id='line_plot', component_property='figure'),
             Input(component_id='my_input', component_property='value'),
             Input(component_id='dropdown', component_property='value'),
             Input(component_id='calculation', component_property='value'),
             Input(component_id='displayed_fields', component_property='value'),
             Input(component_id='top_n', component_property='value')
             )

# def graph_update(input_value, focus_field_value, display_field, calculation_focus):
def graph_update(input_value, focus_field_value, calc_value, display_field, top_n_val):
#     print(input_value)
#     print(type(input_value))
#     print(focus_field_value)
#     print(type(focus_field_value))
#     print(display_field)
#     print(type(display_field))
#     print(top_n_val)
#     print(type(top_n_val))
    cols=['made_field_goals', 'made_three_point_field_goals','made_free_throws', 
      'total_rebounds', 'offensive_rebounds', 'defensive_rebounds', 'assists', 
      'steals', 'blocks', 'turnovers', 'personal_fouls', 'points_scored', 'minutes_played']
    ####
    days_ago=int(input_value)
    today=dt.date.today()
    days_back=today-dt.timedelta(days=days_ago)

#     focus_value=focus_field_value
    
#     calc=calc_value
    
    if top_n_val=='':
        player_sample=5
    else:
        player_sample=int(top_n_val)

    
    df_query=fa_df.query("date >= @days_back")
    output=df_query.groupby(['name'])[cols].agg(calc_value).reset_index().sort_values(by=[focus_field_value],ascending=False).head(player_sample)
    
    output.set_index(['name'], inplace=True, drop=True, append=False)
    output.reset_index(inplace=False)
#     short_df = output.iloc[:,0:len(output)]
# #     short_df = output.iloc[:,0:len(output)-1]
    
    fig=px.imshow(output[display_field], text_auto=True) # was short_df
    fig.update_xaxes(side='top')
#     fig.title(f'Calculation - {calculation_focus}, focused on {focus_field_value}, for the past {input_value}(s)')
    fig.layout.height=800
    fig.layout.width=800
    
    return fig


In [1]:
if __name__ == '__main__': 
    app.run(port=8005)

In [1]:
# exit